<a href="https://colab.research.google.com/github/mtszkw/kaggle-lish-moa/blob/main/Copy_of_kaggle_lish_moa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Envrironment

In [ ]:
input_dir = '/content/'
# input_dir = '../input/lish-moa'

In [ ]:
# https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter

import os
import sys

!pip -q install --upgrade kaggle

%env KAGGLE_USERNAME=mtszkw
%env KAGGLE_KEY=4542560feb261d28dd30710ffc95cb41

# Dataset
# https://www.kaggle.com/c/lish-moa
!kaggle competitions download -c lish-moa

# iterative-stratification
# https://www.kaggle.com/yasufuminakama/iterative-stratification
!kaggle datasets download -d yasufuminakama/iterative-stratification

!unzip -q '/content/*.zip'

for x in os.listdir('/content'):
    if x.endswith(".zip"):
        os.remove(os.path.join('/content', x))

os.listdir('/content')

env: KAGGLE_USERNAME=mtszkw
env: KAGGLE_KEY=4542560feb261d28dd30710ffc95cb41
  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 83.7MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 74.4MB/s]
  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 35.9MB/s]
 88% 48.0M/54.8M [00:00<00:00, 217MB/s]
100% 54.8M/54.8M [00:00<00:00, 218MB/s]
  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 69.9MB/s]
  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 37.3MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 9.20MB/s]

6 archives were successfully processed.


['.config',
 'train_features.csv',
 'train_targets_scored.csv',
 'iterative-stratification-master',
 'sample_submission.csv',
 'train_targets_nonscored.csv',
 'test_features.csv',
 'train_drug.csv',
 'sample_data']

In [ ]:
import argparse
import numpy as np
import pandas as pd

# from sklearn.metrics import log_loss
# from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip install --upgrade wandb
import wandb
!wandb login 733bc0d427366e6b56678aae411349e0381c2625

!pip -q install pytorch-lightning
import pytorch_lightning as pl

!pip -q install '/content/iterative-stratification-master'
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

     |████████████████████████████████| 1.8MB 6.0MB/s 
     |████████████████████████████████| 163kB 42.7MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 133kB 40.4MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=4089206b60fc78c5fd0a93823fbde552679c9e705ac5e3a84198509c916e3880
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=44f6fe40b5a5d10bf82154914094188aa0982157b11ba22b4b171f3c62312341
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=9a77aebdb0348cebf0e0bcedadcf53d9046bb220de2

### Read data

In [ ]:
train_features = pd.read_csv(os.path.join(input_dir, 'train_features.csv'))
train_targets_scored = pd.read_csv(os.path.join(input_dir, 'train_targets_scored.csv'))

test_features = pd.read_csv(os.path.join(input_dir, 'test_features.csv'))

# train_targets_nonscored = pd.read_csv(os.path.join(input_dir, 'train_targets_nonscored.csv'))
# submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))

print(f'Train: {train_features.shape}, test: {test_features.shape}')

Train: (23814, 876), test: (3982, 876)


## Training

### Define model

In [ ]:
class MoANeuralNetwork(pl.LightningModule):
    def __init__(self, input_size: int, output_size: int, cfg: argparse.Namespace, fold_idx: int ):
        super().__init__()
        self.cfg = cfg
        self.fold_idx = fold_idx
        self.net = self.get_model(input_size, output_size)

    def get_model(self, input_size, output_size):
        return nn.Sequential(
            nn.Linear(input_size, cfg.hidden_size),
            nn.BatchNorm1d(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.PReLU(),
            nn.Linear(cfg.hidden_size, cfg.hidden_size),
            nn.BatchNorm1d(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.PReLU(),
            nn.Linear(cfg.hidden_size, output_size))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=0.01, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        return [optimizer], [scheduler]

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = torch.nn.BCEWithLogitsLoss()(y_hat, y)

        self.log_dict({f'train/loss_{self.fold_idx}': loss})
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        val_loss = torch.nn.BCEWithLogitsLoss()(y_hat, y)

        self.log_dict({f'val/loss_{self.fold_idx}': val_loss})
        return val_loss


### Preprocess data

In [ ]:
# ref: https://www.kaggle.com/c/lish-moa/discussion/180165
# check if labels for 'ctl_vehicle' are all 0.
train_data = train_features.merge(train_targets_scored, on='sig_id')
train_data = train_data[train_data['cp_type']!='ctl_vehicle'].reset_index(drop=True)

test_data  = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

print(f'Train data: {train_data.shape}, test data: {test_data.shape}')

Train data: (21948, 1082), test data: (3624, 876)


In [ ]:
target_cols = [c for c in train_targets_scored.columns if c not in ['sig_id']]

len(target_cols)

206

In [ ]:
def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 3, 'D2': 4})
    return df

train_data = cate2num(train_data)
test_data  = cate2num(test_data)
feature_cols = [c for c in train_data.columns if train_data.dtypes[c] != 'object']
feature_cols = [c for c in feature_cols if c not in target_cols]

len(feature_cols)

874

### Train model

In [ ]:
cfg = argparse.Namespace()
cfg.n_splits = 5
cfg.batch_size = 32
cfg.dropout = .5
cfg.hidden_size = 512

In [ ]:
logger = pl.loggers.WandbLogger(project='kaggle-lish-moa')
logger.log_hyperparams(cfg)

In [ ]:
folds = train_data.copy()
k_fold = MultilabelStratifiedKFold(n_splits=cfg.n_splits, shuffle=False, random_state=42)
for n, (train_index, valid_index) in enumerate(k_fold.split(folds, folds[target_cols])):

    df_train = folds.iloc[train_index]
    df_valid = folds.iloc[valid_index]

    train_y = torch.tensor(df_train[target_cols].values.astype(np.float32))
    train_x = torch.tensor(df_train[feature_cols].values.astype(np.float32)) 
    train_tensor = torch.utils.data.TensorDataset(train_x, train_y) 

    valid_y = torch.tensor(df_valid[target_cols].values.astype(np.float32))
    valid_x = torch.tensor(df_valid[feature_cols].values.astype(np.float32)) 
    valid_tensor = torch.utils.data.TensorDataset(valid_x, valid_y) 

    train_dl = torch.utils.data.DataLoader(train_tensor, shuffle=False, batch_size=cfg.batch_size)    
    valid_dl = torch.utils.data.DataLoader(valid_tensor, shuffle=False, batch_size=cfg.batch_size)

    trainer = pl.Trainer(gpus=1, max_epochs=5, logger=logger)

    lit_module = MoANeuralNetwork(train_x.shape[1], train_y.shape[1], cfg, n)
    trainer.fit(lit_module, train_dl, valid_dl)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 


GPU available: True, used: True


TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 
